- what type of documents does it contain?
- how many documents are there?
- calculate and visualise some simple statistics for the collection, e.g. the average document length,
the average vocabulary size, etc.

## Import the dataset

In [1]:
import json

f = open(r'/kaggle/input/wizard-of-wikipedia/train.json', 'r')
dataset = json.loads(f.read())

# Skip this code if you want to use entire dataset
dataset = dataset[:4000]



In [2]:
# Inspect the format of the imported json
#dataset

### What's inside this dataset?

 

PREPEND EACH SENTENCE WITH ARTICLE TITLE: "We retrieve the top 7 articles (first paragraph only) for each lookup and then flatten all the results into separate sentences (i.e. remove the organization of sentences belonging to articles), but prepend every sentence with its article title. In this way the candidates mc1 , . . . , mcK given to the neural model in the next stage can be attended to independently without having to deal with hierarchical issues."

DIALOGUE CONTEXT: "it is given the current dialogue context x1 , . . . , xt of t dialogue turns, where x1 is al- ways the initial starting topic (e.g. “Kurt Cobain”), and the remaining turns swap between the two speakers." So, x1 is the topic and x2 will be the first generated text, x2 the answer and so on...
THE GOAL AT EACH STAGE IS TO OUTPUT THE NEXT UTTERANCE xt+1.

Now, let's try extracting the CHOSEN TOPIC + TEXT MESSAGE to predict which of the RETRIEVED PASSAGES is the best one.

## Building the structure

In [3]:
#dataset=dataset[:15]

In [4]:
def create_passages_struct(dataset):
    structure = []
    for conversation in dataset:
        dialog_structure = []
        for dialog in conversation['dialog']:
            dialog_entry = [f"{list(passage.keys())[0]}: {sentence}" for passage in dialog['retrieved_passages'] for sentence in list(passage.values())[0]]
            dialog_structure.append(dialog_entry)
        structure.append(dialog_structure)
    return structure

passages_struct = create_passages_struct(dataset)             

In [5]:
#passages_struct[:1]

In [6]:
def create_dialog_context_struct(dataset):
    transformed_data = []
    for item in dataset:
        dialog = item['dialog']
        text_list = []
        flag = 0
        for utterance in dialog:
            if flag == 0: 
                text_list.append('Person A: '+utterance['text'])
                flag = 1
            else: 
                text_list.append('Person B: '+utterance['text'])
                flag = 0
        transformed_data.append(text_list)
    return transformed_data


dialog_context_struct = create_dialog_context_struct(dataset)

Splitting the dialog_context_struct in a training set and validation set

In [7]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(dialog_context_struct, test_size=0.2, random_state=42)

In [8]:
#dialog_context_struct[0:2]

In [9]:
def build_batch(d_c_struct, p_struct, conv, turn):
    length = len(d_c_struct[conv])
    if turn >= length-1: 
        return d_c_struct[conv][length-1], ''.join(d_c_struct[conv][:length-1]), p_struct[conv][length-1]
    else: 
        return d_c_struct[conv][turn+1], ''.join(d_c_struct[conv][0:turn+1]), p_struct[conv][turn]

#conv = 0
#turn = 10

#expected_answer, dialog_context, retrieved_passages = build_batch(dialog_context_struct, passages_struct, conv, turn)

#print(dialog_context)
#print(retrieved_passages)
#print(expected_answer)


In [10]:

#print(tdialog_context_struct))
#print(type(dialog_context_struct[0]))

In [11]:
#input_text = f"Given the following passage, continue the related conversation.\n\nPassage:\n\n{passage}\n\nCONVERSATION: {dialog_context}"

## Tutotial 9 adaptation

Building the dictionary stuctures for the model

Encoder

In [12]:
!pip -q install transformers==4.22.2
!pip -q install -U sentence-transformers

Compute the embeddings

In [13]:
#passages_struct[0][0]

In [14]:
#embedded_sentences[0][0][0].shape

Decoder

In [15]:
pip freeze | grep transformers

sentence-transformers==2.2.2
transformers==4.22.2
Note: you may need to restart the kernel to use updated packages.


In [16]:
!pip install accelerate

Preparation of train and valid sets for fine tuning

In [17]:
import wandb

wandb.login(key="3b6a39543333ed99129ca04698a32714e7f91fc4")
#wandb.login(key=[3b6a39543333ed99129ca04698a32714e7f91fc4])

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [18]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_id = 'gpt2'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")


model.to(device)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [19]:
tokenizer.pad_token = tokenizer.eos_token

In [20]:
def build_strings(d_c_struct, eos_token):
    strings = []
    for conversation in d_c_struct:
        strings.append(eos_token.join(conversation))
    return strings

test = build_strings(dialog_context_struct, tokenizer.eos_token)

In [21]:
# Split dataset into train and valid sets, then build the dialog context strings

training_data_str = build_strings(test, tokenizer.eos_token)
validation_data_str = build_strings(valid, tokenizer.eos_token)


Building the fine tuning dataset

In [22]:
from datasets import Dataset

train_data = Dataset.from_dict({'text': training_data_str})
valid_data = Dataset.from_dict({'text': validation_data_str})

In [23]:
from datasets import DatasetDict

data = DatasetDict()
data['train'] = train_data
data['validation'] = valid_data
data['test'] = valid_data

Tokenization of inputs

In [24]:
def tokenize_function(examples):
    input_encodings = tokenizer(examples["text"], padding=True, truncation=True)
    sample = {
        'input_ids': input_encodings.input_ids
    }
    return sample

tokenized_data = data.map(tokenize_function, batched=True)

# TO DO: CACHE THIS tokenized_data SO YOU DON'T HAVE TO RECALCULATE IT EVERY TIME !!!

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [25]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [26]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "ash_ketchum", 
    #evaluation_strategy="steps",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=7,
    learning_rate=6.25e-5,
    lr_scheduler_type="linear",
    save_strategy="no"
    
)

In [27]:
optim = torch.optim.AdamW(model.parameters(),lr=5e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optim,20,eta_min=1e-7)

In [28]:
!pip install -U accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0


In [29]:
#!pip -q install transformers==4.22.2

In [30]:
from transformers import TrainingArguments, Trainer

trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=tokenized_data['train'], 
    eval_dataset=tokenized_data['validation'],
    data_collator=data_collator,
)

In [31]:
import gc
torch.cuda.empty_cache()
gc.collect()

76

In [32]:
import gc
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()
gc.collect()
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4000
  Num Epochs = 7
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 7000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: msanjaysivakumar (mercurial). Use `wandb login --relogin` to force relogin
wandb: Trac

Step,Training Loss
500,1.552200
1000,1.203200
1500,1.107700
2000,1.067400
2500,1.020600
3000,0.999700
3500,0.971200
4000,0.960100
4500,0.940500
5000,0.931500




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=7000, training_loss=1.0275461948939733, metrics={'train_runtime': 5273.8227, 'train_samples_per_second': 5.309, 'train_steps_per_second': 1.327, 'total_flos': 1.4632353792e+16, 'train_loss': 1.0275461948939733, 'epoch': 7.0})

In [33]:
from datetime import datetime

checkpoint_path = f"persona_chat_fine_tuning_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}"
tokenizer.save_pretrained(checkpoint_path)
model.save_pretrained(checkpoint_path)
print(f"Checkpoint saved at: \'{checkpoint_path}\'")

tokenizer config file saved in persona_chat_fine_tuning_2023_05_29_00_18_06/tokenizer_config.json
Special tokens file saved in persona_chat_fine_tuning_2023_05_29_00_18_06/special_tokens_map.json
Configuration saved in persona_chat_fine_tuning_2023_05_29_00_18_06/config.json
Model weights saved in persona_chat_fine_tuning_2023_05_29_00_18_06/pytorch_model.bin


Checkpoint saved at: 'persona_chat_fine_tuning_2023_05_29_00_18_06'


In [34]:
!zip -r /kaggle/working/output.zip /kaggle/working/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/persona_chat_fine_tuning_2023_05_29_00_18_06/ (stored 0%)
  adding: kaggle/working/persona_chat_fine_tuning_2023_05_29_00_18_06/special_tokens_map.json (deflated 60%)
  adding: kaggle/working/persona_chat_fine_tuning_2023_05_29_00_18_06/pytorch_model.bin (deflated 16%)
  adding: kaggle/working/persona_chat_fine_tuning_2023_05_29_00_18_06/tokenizer.json (deflated 72%)
  adding: kaggle/working/persona_chat_fine_tuning_2023_05_29_00_18_06/config.json (deflated 52%)
  adding: kaggle/working/persona_chat_fine_tuning_2023_05_29_00_18_06/vocab.json (deflated 59%)
  adding: kaggle/working/persona_chat_fine_tuning_

In [35]:
from IPython.display import FileLink 
FileLink(r'output.zip')

/kaggle/working/output.zip

In [36]:
#print(len(tokenized_data['train']))
#print(len(tokenized_data['validation']))


The resulting sentence is not the same we find in the dataset! Maybe we could find a way to fine-tune this BERT encoder, using as batches this sequences, and as expected label, the sentence it should instead find. For example, in this case, the index should have been 4.
The problem of building this dataset is that we need to specifically build it by looking at the label... or maybe find a way to reduce it to a classification problem? Find the better frase between 7? I don't know...

Part 1: Knowledge Attention

We need to use an attention dot-product mechanism to decide which sentence should be used to generate the next utterance. To do so, each sentence is encoded INDIPENDENTLY with a transformer encoder, and THE SAME TRANSFORMER IS USED TO ENCODE THE DIALOGUE CONTEXT x.
We then perform standard dot-product attention between the sentence candidates and the dialogue context.

## Transformer transfer learning


## Transformer

### Embedding and positional encoding


### Embedding and positional encoding of input

### Transformer encoder

Due to the fact that we are working with knowledge extracted from Wikipedia, we can use a Wikipedia pre-trained embedding such as glove-wiki-gigaword-50. Let's see how the embedding of wikipedia compares with the embedding of the conversational text of the dataset.

### Comparing the two embeddings

#### Obviously the conversational embedding yealds better results with terms such as 'haha', 'lol', while more complex words like 'axolotl' don't even exist in our conversational vocabulary.